Continue based on xgboost notebook

use rolling approach - build a model that can  predict the classes for one day at a time. I recommend a rolling forward process for training and testing. Like 5-12 days for training and 1 day for testing.

rolling window 5 + 1

same data

use claude and confusion metric 

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
optuna.logging.set_verbosity(optuna.logging.WARNING)
pio.renderers.default = "notebook"
import warnings
warnings.filterwarnings('ignore')

def load_and_preprocess_data(file_path):
    """Load and preprocess the data."""
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Convert open_time to datetime
    df['Open time'] = pd.to_datetime(df['Open time'])
    
    # Get feature columns (excluding open_time and target)
    feature_cols = [col for col in df.columns if col.startswith('feature_')]
    
    return df, feature_cols

def train_test_split_by_date(df, train_days, test_days, start_idx):
    """Split data into training and test sets based on dates."""
    all_dates = df['Open time'].dt.date.unique()
    
    if start_idx + train_days + test_days > len(all_dates):
        return None, None, None, None
    
    train_dates = all_dates[start_idx:start_idx + train_days]
    test_dates = all_dates[start_idx + train_days:start_idx + train_days + test_days]
    
    train_mask = df['Open time'].dt.date.isin(train_dates)
    test_mask = df['Open time'].dt.date.isin(test_dates)
    
    return df[train_mask], df[test_mask], train_dates, test_dates

def evaluate_predictions(y_true, y_pred):
    """Calculate regression metrics."""
    return {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100,  # Added small constant to avoid division by zero
        'direction_accuracy': np.mean((y_true * y_pred) > 0)  # Direction prediction accuracy
    }

def create_model(trial):
    """Create a model with parameters suggested by Optuna."""
    return XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 300),
        max_depth=trial.suggest_int('max_depth', 2, 8),
        learning_rate=trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        min_child_weight=trial.suggest_float('min_child_weight', 1, 7),
        subsample=trial.suggest_float('subsample', 0.6, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
        gamma=trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        reg_alpha=trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        reg_lambda=trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        random_state=42
    )

def optimize_parameters(train_df, val_df, feature_cols, n_trials=50):
    """Optimize hyperparameters using Optuna."""
    X_train = train_df[feature_cols]
    y_train = train_df['target']
    X_val = val_df[feature_cols]
    y_val = val_df['target']
    
    def objective(trial):
        model = create_model(trial)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=20,
            verbose=False
        )
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    
    best_model = create_model(study.best_trial)
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=20,
        verbose=False
    )
    
    return best_model, study

def plot_optimization_results(study, window_idx):
    """Plot optimization results using Plotly."""
    # Plot optimization history
    history_fig = go.Figure()
    history_data = {
        'number': list(range(len(study.trials))),
        'value': [t.value for t in study.trials]
    }
    
    history_fig.add_trace(
        go.Scatter(
            x=history_data['number'],
            y=history_data['value'],
            mode='lines+markers',
            name='RMSE'
        )
    )
    
    history_fig.update_layout(
        title=f'Optimization History - Window {window_idx}',
        xaxis_title='Trial',
        yaxis_title='RMSE',
        height=400
    )
    
    # Plot parameter importances
    importance = optuna.importance.get_param_importances(study)
    importance_df = pd.DataFrame({
        'parameter': list(importance.keys()),
        'importance': list(importance.values())
    }).sort_values('importance', ascending=True)
    
    param_fig = go.Figure()
    param_fig.add_trace(
        go.Bar(
            x=importance_df['importance'],
            y=importance_df['parameter'],
            orientation='h'
        )
    )
    
    param_fig.update_layout(
        title=f'Parameter Importances - Window {window_idx}',
        xaxis_title='Importance',
        yaxis_title='Parameter',
        height=400
    )
    
    return history_fig, param_fig

def train_and_evaluate_window(train_df, test_df, feature_cols, window_idx):
    """Train XGBoost and evaluate for a single window with hyperparameter optimization."""
    global current_feature_importance
    
    # Split train data into train and validation for optimization
    train_dates = train_df['Open time'].dt.date.unique()
    val_size = max(1, len(train_dates) // 5)  # 20% of training data for validation
    
    val_dates = train_dates[-val_size:]
    train_dates = train_dates[:-val_size]
    
    train_mask = train_df['Open time'].dt.date.isin(train_dates)
    val_mask = train_df['Open time'].dt.date.isin(val_dates)
    
    opt_train_df = train_df[train_mask]
    val_df = train_df[val_mask]
    
    # Optimize parameters
    best_model, study = optimize_parameters(opt_train_df, val_df, feature_cols)
    
    # Create optimization plots
    # history_fig, param_fig = plot_optimization_results(study, window_idx)
    # history_fig.write_html(f"optimization_history_window_{window_idx}.html")
    # param_fig.write_html(f"parameter_importance_window_{window_idx}.html")
    
    # Save best parameters
    best_params = study.best_params
    
    # Make predictions on test set
    predictions = best_model.predict(test_df[feature_cols])
    
    # Calculate metrics
    metrics = evaluate_predictions(test_df['target'], predictions)
    
    # Store feature importance
    global current_feature_importance
    current_feature_importance = best_model.feature_importances_
    
    return metrics, predictions, best_params

def rolling_forward_validation(df, feature_cols, train_days=5, test_days=1):
    """Perform rolling forward validation."""
    results = []
    start_idx = 0
    maxcnt = 0
    while True or maxcnt < 5:
        maxcnt += 1
        # Get train/test split for current window
        train_df, test_df, train_dates, test_dates = train_test_split_by_date(
            df, train_days, test_days, start_idx
        )
        
        if train_df is None:  # No more data
            break
            
        # Train and evaluate
        metrics, predictions, best_params = train_and_evaluate_window(
            train_df, test_df, feature_cols, start_idx
        )
        
        # Store results
        results.append({
            'train_start': min(train_dates),
            'train_end': max(train_dates),
            'test_start': min(test_dates),
            'test_end': max(test_dates),
            'metrics': metrics,
            'predictions': predictions,
            'best_params': best_params,
            'actual_values': test_df['target'].values,
            'dates': test_df['Open time'].values
        })
        
        start_idx += test_days
        
    return results

def calculate_aggregated_metrics(results):
    """Calculate aggregated metrics across all windows."""
    all_predictions = np.concatenate([r['predictions'] for r in results])
    all_actuals = np.concatenate([r['actual_values'] for r in results])
    
    # Calculate overall metrics
    overall_metrics = evaluate_predictions(all_actuals, all_predictions)
    
    # Calculate per-window metrics
    window_metrics = pd.DataFrame([
        {
            'test_date': r['test_start'],
            **r['metrics']
        } for r in results
    ])
    
    return {
        'overall_metrics': overall_metrics,
        'window_metrics': window_metrics
    }

def plot_metrics_over_time(window_metrics_df):
    """Plot performance metrics over time using Plotly."""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Error Metrics Over Time', 'R² and Direction Accuracy Over Time'),
        vertical_spacing=0.15
    )

    # Plot error metrics
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['rmse'],
                  name="RMSE", mode='lines+markers'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['mae'],
                  name="MAE", mode='lines+markers'),
        row=1, col=1
    )
    
    # Plot R² and direction accuracy
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['r2'],
                  name="R²", mode='lines+markers'),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['direction_accuracy'],
                  name="Direction Accuracy", mode='lines+markers'),
        row=2, col=1
    )

    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Model Performance Metrics Over Time",
        hovermode='x unified'
    )
    fig.update_xaxes(title_text="Test Date", row=1, col=1)
    fig.update_xaxes(title_text="Test Date", row=2, col=1)
    fig.update_yaxes(title_text="Error", row=1, col=1)
    fig.update_yaxes(title_text="Score", row=2, col=1)
    
    fig.write_html("metrics_over_time.html")
    return fig

def plot_predictions_vs_actual(results):
    """Plot predictions vs actual values using Plotly."""
    all_dates = np.concatenate([r['dates'] for r in results])
    all_predictions = np.concatenate([r['predictions'] for r in results])
    all_actuals = np.concatenate([r['actual_values'] for r in results])
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(x=all_dates, y=all_actuals,
                  name="Actual Returns", mode='lines',
                  line=dict(color='blue'))
    )
    fig.add_trace(
        go.Scatter(x=all_dates, y=all_predictions,
                  name="Predicted Returns", mode='lines',
                  line=dict(color='red'))
    )
    
    fig.update_layout(
        title='Predicted vs Actual Returns Over Time',
        xaxis_title='Date',
        yaxis_title='Returns',
        hovermode='x unified',
        height=600
    )
    
    fig.write_html("predictions_vs_actual.html")
    return fig

def plot_residuals(results):
    """Plot residuals analysis using Plotly."""
    all_predictions = np.concatenate([r['predictions'] for r in results])
    all_actuals = np.concatenate([r['actual_values'] for r in results])
    residuals = all_actuals - all_predictions
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Residuals Distribution', 'Residuals vs Predicted'),
        specs=[[{"type": "histogram"}, {"type": "scatter"}]]
    )
    
    # Residuals distribution
    fig.add_trace(
        go.Histogram(x=residuals, name="Residuals",
                    nbinsx=50),
        row=1, col=1
    )
    
    # Residuals vs Predicted
    fig.add_trace(
        go.Scatter(x=all_predictions, y=residuals,
                  mode='markers', name="Residuals vs Predicted",
                  marker=dict(size=5, color='blue', opacity=0.5)),
        row=1, col=2
    )
    
    fig.update_layout(
        height=400,
        title_text="Residuals Analysis",
        showlegend=False
    )
    
    fig.write_html("residuals_analysis.html")
    return fig

def plot_feature_importance(feature_cols, importance_scores, top_n=10):
    """Plot feature importance scores using Plotly."""
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance_scores
    }).sort_values('importance', ascending=True)
    
    importance_df = importance_df.tail(top_n)
    
    fig = go.Figure(go.Bar(
        x=importance_df['importance'],
        y=importance_df['feature'],
        orientation='h'
    ))
    
    fig.update_layout(
        title=f'Top {top_n} Most Important Features',
        xaxis_title='Importance Score',
        yaxis_title='Feature',
        height=max(400, top_n * 25),
        showlegend=False,
        yaxis={'categoryorder': 'total ascending'}
    )
    
    fig.write_html("feature_importance.html")
    return fig

# Example usage:
if __name__ == "__main__":
    # Load and preprocess data
    df, feature_cols = load_and_preprocess_data('model_data.csv')
    
    # Perform rolling forward validation
    results = rolling_forward_validation(df, feature_cols, train_days=5, test_days=1)
    
    # Calculate aggregated metrics
    metrics = calculate_aggregated_metrics(results)
    
    # Print overall results
    print("\nOverall Metrics:")
    for metric, value in metrics['overall_metrics'].items():
        print(f"{metric}: {value:.4f}")
    
    print("\nPer-window Metrics Summary:")
    print(metrics['window_metrics'].describe())
    
    # Create and save visualizations
    print("\nGenerating and saving interactive plots...")
    
    # 1. Metrics over time
    metrics_plot = plot_metrics_over_time(metrics['window_metrics'])
    
    # 2. Predictions vs Actual
    predictions_plot = plot_predictions_vs_actual(results)
    
    # 3. Residuals analysis
    residuals_plot = plot_residuals(results)
    
    # 4. Feature importance
    if 'current_feature_importance' in globals():
        feature_imp_plot = plot_feature_importance(
            feature_cols, 
            current_feature_importance
        )
    
    print("Interactive HTML plots have been saved to the current directory.")

/Users/davidbrazda/Documents/Development/python/mlroom/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
optuna.logging.set_verbosity(optuna.logging.WARNING)
pio.renderers.default = "notebook"
import warnings
warnings.filterwarnings('ignore')

def load_and_preprocess_data(file_path):
    """Load and preprocess the data."""
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Convert open_time to datetime
    df['open_time'] = pd.to_datetime(df['open_time'])
    
    # Get feature columns (excluding open_time and target)
    feature_cols = [col for col in df.columns if col.startswith('feature_')]
    
    return df, feature_cols

def train_test_split_by_date(df, train_days, test_days, start_idx):
    """Split data into training and test sets based on dates."""
    all_dates = df['open_time'].dt.date.unique()
    
    if start_idx + train_days + test_days > len(all_dates):
        return None, None, None, None
    
    train_dates = all_dates[start_idx:start_idx + train_days]
    test_dates = all_dates[start_idx + train_days:start_idx + train_days + test_days]
    
    train_mask = df['open_time'].dt.date.isin(train_dates)
    test_mask = df['open_time'].dt.date.isin(test_dates)
    
    return df[train_mask], df[test_mask], train_dates, test_dates

def evaluate_predictions(y_true, y_pred):
    """Calculate regression metrics."""
    return {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100,  # Added small constant to avoid division by zero
        'direction_accuracy': np.mean((y_true * y_pred) > 0)  # Direction prediction accuracy
    }

def create_model(trial):
    """Create a model with parameters suggested by Optuna."""
    return XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 300),
        max_depth=trial.suggest_int('max_depth', 2, 8),
        learning_rate=trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        min_child_weight=trial.suggest_float('min_child_weight', 1, 7),
        subsample=trial.suggest_float('subsample', 0.6, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
        gamma=trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        reg_alpha=trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        reg_lambda=trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        random_state=42
    )

def optimize_parameters(train_df, val_df, feature_cols, n_trials=50):
    """Optimize hyperparameters using Optuna."""
    X_train = train_df[feature_cols]
    y_train = train_df['target']
    X_val = val_df[feature_cols]
    y_val = val_df['target']
    
    def objective(trial):
        model = create_model(trial)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=20,
            verbose=False
        )
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        return rmse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    
    best_model = create_model(study.best_trial)
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=20,
        verbose=False
    )
    
    return best_model, study

def plot_optimization_results(study, window_idx):
    """Plot optimization results using Plotly."""
    # Plot optimization history
    history_fig = go.Figure()
    history_data = {
        'number': list(range(len(study.trials))),
        'value': [t.value for t in study.trials]
    }
    
    history_fig.add_trace(
        go.Scatter(
            x=history_data['number'],
            y=history_data['value'],
            mode='lines+markers',
            name='RMSE'
        )
    )
    
    history_fig.update_layout(
        title=f'Optimization History - Window {window_idx}',
        xaxis_title='Trial',
        yaxis_title='RMSE',
        height=400
    )
    
    # Plot parameter importances
    importance = optuna.importance.get_param_importances(study)
    importance_df = pd.DataFrame({
        'parameter': list(importance.keys()),
        'importance': list(importance.values())
    }).sort_values('importance', ascending=True)
    
    param_fig = go.Figure()
    param_fig.add_trace(
        go.Bar(
            x=importance_df['importance'],
            y=importance_df['parameter'],
            orientation='h'
        )
    )
    
    param_fig.update_layout(
        title=f'Parameter Importances - Window {window_idx}',
        xaxis_title='Importance',
        yaxis_title='Parameter',
        height=400
    )
    
    return history_fig, param_fig

def train_and_evaluate_window(train_df, test_df, feature_cols, window_idx):
    """Train XGBoost and evaluate for a single window with hyperparameter optimization."""
    global current_feature_importance
    
    # Split train data into train and validation for optimization
    train_dates = train_df['open_time'].dt.date.unique()
    val_size = max(1, len(train_dates) // 5)  # 20% of training data for validation
    
    val_dates = train_dates[-val_size:]
    train_dates = train_dates[:-val_size]
    
    train_mask = train_df['open_time'].dt.date.isin(train_dates)
    val_mask = train_df['open_time'].dt.date.isin(val_dates)
    
    opt_train_df = train_df[train_mask]
    val_df = train_df[val_mask]
    
    # Optimize parameters
    best_model, study = optimize_parameters(opt_train_df, val_df, feature_cols)
    
    # Create optimization plots
    history_fig, param_fig = plot_optimization_results(study, window_idx)
    history_fig.write_html(f"optimization_history_window_{window_idx}.html")
    param_fig.write_html(f"parameter_importance_window_{window_idx}.html")
    
    # Save best parameters
    best_params = study.best_params
    
    # Make predictions on test set
    predictions = best_model.predict(test_df[feature_cols])
    
    # Calculate metrics
    metrics = evaluate_predictions(test_df['target'], predictions)
    
    # Store feature importance
    global current_feature_importance
    current_feature_importance = best_model.feature_importances_
    
    return metrics, predictions, best_params

def rolling_forward_validation(df, feature_cols, train_days=5, test_days=1):
    """Perform rolling forward validation."""
    results = []
    start_idx = 0
    
    while True:
        # Get train/test split for current window
        train_df, test_df, train_dates, test_dates = train_test_split_by_date(
            df, train_days, test_days, start_idx
        )
        
        if train_df is None:  # No more data
            break
            
        # Train and evaluate
        metrics, predictions, best_params = train_and_evaluate_window(
            train_df, test_df, feature_cols, start_idx
        )
        
        # Store results
        results.append({
            'train_start': min(train_dates),
            'train_end': max(train_dates),
            'test_start': min(test_dates),
            'test_end': max(test_dates),
            'metrics': metrics,
            'predictions': predictions,
            'best_params': best_params,
            'actual_values': test_df['target'].values,
            'dates': test_df['open_time'].values
        })
        
        start_idx += test_days
        
    return results

def calculate_aggregated_metrics(results):
    """Calculate aggregated metrics across all windows."""
    all_predictions = np.concatenate([r['predictions'] for r in results])
    all_actuals = np.concatenate([r['actual_values'] for r in results])
    
    # Calculate overall metrics
    overall_metrics = evaluate_predictions(all_actuals, all_predictions)
    
    # Calculate per-window metrics
    window_metrics = pd.DataFrame([
        {
            'test_date': r['test_start'],
            **r['metrics']
        } for r in results
    ])
    
    return {
        'overall_metrics': overall_metrics,
        'window_metrics': window_metrics
    }

def plot_metrics_over_time(window_metrics_df):
    """Plot performance metrics over time using Plotly."""
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Error Metrics Over Time', 'R² and Direction Accuracy Over Time'),
        vertical_spacing=0.15
    )

    # Plot error metrics
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['rmse'],
                  name="RMSE", mode='lines+markers'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['mae'],
                  name="MAE", mode='lines+markers'),
        row=1, col=1
    )
    
    # Plot R² and direction accuracy
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['r2'],
                  name="R²", mode='lines+markers'),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=window_metrics_df['test_date'], y=window_metrics_df['direction_accuracy'],
                  name="Direction Accuracy", mode='lines+markers'),
        row=2, col=1
    )

    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Model Performance Metrics Over Time",
        hovermode='x unified'
    )
    fig.update_xaxes(title_text="Test Date", row=1, col=1)
    fig.update_xaxes(title_text="Test Date", row=2, col=1)
    fig.update_yaxes(title_text="Error", row=1, col=1)
    fig.update_yaxes(title_text="Score", row=2, col=1)
    
    fig.write_html("metrics_over_time.html")
    return fig

def plot_predictions_vs_actual(results):
    """Plot predictions vs actual values using Plotly."""
    all_dates = np.concatenate([r['dates'] for r in results])
    all_predictions = np.concatenate([r['predictions'] for r in results])
    all_actuals = np.concatenate([r['actual_values'] for r in results])
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(x=all_dates, y=all_actuals,
                  name="Actual Returns", mode='lines',
                  line=dict(color='blue'))
    )
    fig.add_trace(
        go.Scatter(x=all_dates, y=all_predictions,
                  name="Predicted Returns", mode='lines',
                  line=dict(color='red'))
    )
    
    fig.update_layout(
        title='Predicted vs Actual Returns Over Time',
        xaxis_title='Date',
        yaxis_title='Returns',
        hovermode='x unified',
        height=600
    )
    
    fig.write_html("predictions_vs_actual.html")
    return fig

def plot_residuals(results):
    """Plot residuals analysis using Plotly."""
    all_predictions = np.concatenate([r['predictions'] for r in results])
    all_actuals = np.concatenate([r['actual_values'] for r in results])
    residuals = all_actuals - all_predictions
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Residuals Distribution', 'Residuals vs Predicted'),
        specs=[[{"type": "histogram"}, {"type": "scatter"}]]
    )
    
    # Residuals distribution
    fig.add_trace(
        go.Histogram(x=residuals, name="Residuals",
                    nbinsx=50),
        row=1, col=1
    )
    
    # Residuals vs Predicted
    fig.add_trace(
        go.Scatter(x=all_predictions, y=residuals,
                  mode='markers', name="Residuals vs Predicted",
                  marker=dict(size=5, color='blue', opacity=0.5)),
        row=1, col=2
    )
    
    fig.update_layout(
        height=400,
        title_text="Residuals Analysis",
        showlegend=False
    )
    
    fig.write_html("residuals_analysis.html")
    return fig

def plot_feature_importance(feature_cols, importance_scores, top_n=10):
    """Plot feature importance scores using Plotly."""
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance_scores
    }).sort_values('importance', ascending=True)
    
    importance_df = importance_df.tail(top_n)
    
    fig = go.Figure(go.Bar(
        x=importance_df['importance'],
        y=importance_df['feature'],
        orientation='h'
    ))
    
    fig.update_layout(
        title=f'Top {top_n} Most Important Features',
        xaxis_title='Importance Score',
        yaxis_title='Feature',
        height=max(400, top_n * 25),
        showlegend=False,
        yaxis={'categoryorder': 'total ascending'}
    )
    
    fig.write_html("feature_importance.html")
    return fig

# Example usage:
if __name__ == "__main__":
    # Load and preprocess data
    df, feature_cols = load_and_preprocess_data('model_data.csv')
    
    # Perform rolling forward validation
    results = rolling_forward_validation(df, feature_cols, train_days=5, test_days=1)
    
    # Calculate aggregated metrics
    metrics = calculate_aggregated_metrics(results)
    
    # Print overall results
    print("\nOverall Metrics:")
    for metric, value in metrics['overall_metrics'].items():
        print(f"{metric}: {value:.4f}")
    
    print("\nPer-window Metrics Summary:")
    print(metrics['window_metrics'].describe())
    
    # Create and save visualizations
    print("\nGenerating and saving interactive plots...")
    
    # 1. Metrics over time
    metrics_plot = plot_metrics_over_time(metrics['window_metrics'])
    
    # 2. Predictions vs Actual
    predictions_plot = plot_predictions_vs_actual(results)
    
    # 3. Residuals analysis
    residuals_plot = plot_residuals(results)
    
    # 4. Feature importance
    if 'current_feature_importance' in globals():
        feature_imp_plot = plot_feature_importance(
            feature_cols, 
            current_feature_importance
        )
    
    # 5. Hyperparameter trends
    hyperparameter_plot = plot_hyperparameter_trends(results)
    
    print("\nHyperparameter Optimization Summary:")
    param_stats = pd.DataFrame([r['best_params'] for r in results]).describe()
    print(param_stats)
    
    print("\nInteractive HTML plots have been saved to the current directory.")


Overall Metrics:
rmse: 0.2737
mae: 0.2049
r2: -0.5437
mape: 11041884.0170
direction_accuracy: 0.4804

Per-window Metrics Summary:
             rmse         mae          r2          mape  direction_accuracy
count  216.000000  216.000000  216.000000  2.160000e+02          216.000000
mean     0.249774    0.204853   -1.392180  1.104188e+07            0.480413
std      0.112207    0.097230    3.114653  2.593207e+07            0.149168
min      0.069436    0.056412  -35.917202  7.387412e+01            0.038462
25%      0.169604    0.135464   -1.301220  1.607032e+02            0.384615
50%      0.226256    0.183573   -0.602132  2.333839e+02            0.480769
75%      0.319598    0.264382   -0.204980  2.384562e+06            0.576923
max      0.734586    0.687925    0.551226  1.969526e+08            0.846154

Generating and saving interactive plots...
Interactive HTML plots have been saved to the current directory.
